In [ ]:
import numpy



class FluxLimiter:
    def __init__(self, uminus, ucenter, uplus, flux_limiter):
        self.uminus = uminus
        self.ucenter = ucenter
        self.uplus = uplus

        self.R = numpy.array(
            [(e - c) / (c - w) if c != w else 0 for w, c, e in zip(self.uminus, self.ucenter, self.uplus)]
        )

        self.flux_limiter = flux_limiter

    @staticmethod
    def superbee(R):
        return numpy.minimum(numpy.maximum(1, R), 2, 2 * R)

    @staticmethod
    def vanLeer(R):
        return 2 * R / (R + 1)

    @staticmethod
    def vanAlbada(R):
        return (R * R + R) / (R * R + 1)

    def right_interface_left_limit(self):
        return self.ucenter + 0.5 * self.flux_limiter(self.R) * (self.uplus - self.uminus)

    def left_interface_right_limit(self):
        return self.ucenter - 0.5 * self.flux_limiter(self.R) * (self.uplus - self.uminus)


class RiemannSolver:
    @staticmethod
    def intercell_flux_lax_friedrichs(F, U_L, U_R, S_L, S_R):
        return 0.5 * (F(U_L) + F(U_R)) - 0.5 * max(abs(S_L), abs(S_R)) * (U_R - U_L)

    @staticmethod
    def intercell_flux_hll(F, U_L, U_R, S_L, S_R):
        if 0 <= S_L:
            return F(U_L)
        elif S_L <= 0 <= S_R:
            return (S_R * F(U_L) - S_L * F(U_R) + S_L * S_R * (U_R - U_L)) / (S_R - S_L)
        elif S_R <= 0:
            return F(U_R)
        else:
            raise ValueError()

    @staticmethod
    def intercell_flux(F, U_L, U_R, S_L, S_R):
        return RiemannSolver.intercell_flux_hll(F, U_L, U_R, S_L, S_R)


arXiv:1910.04975v2 (NOTE: g > 0)
$$
\begin{align}
    z_w &= z_b + h\\
    \partial_t(h) + \partial_0(h u_0) + \partial_1(h u_1) &= 0\\
    \partial_t(h u_0) + \partial_0(h u_0 u_0) + \partial_1(h u_0 u_1) + \partial_0(\frac{1}{2} g h^2) + g h\partial_0(z_b) + C_f |u| u_0 &= 0\\
    \partial_t(h u_1) + \partial_0(h u_1 u_0) + \partial_1(h u_1 u_1) + \partial_1(\frac{1}{2} g h^2) + g h\partial_1(z_b) + C_f |u| u_1 &= 0\\
\end{align}
$$

$$
Q = 
\left[\begin{matrix}
h\\
h u_0\\
h u_1\\
\partial_0(z_b)\\
\partial_1(z_b)\\
g\\
C_f\\
\end{matrix}\right]\\
$$

In [ ]:
import findiff
import numpy
import numpy.linalg
import scipy.signal

class SWE2D:
    dry_water = 1e-2
    speed = 1481

    def __init__(
        self,
        *,
        cfl,
        grid,
        bottom,
        water,
        velocity_0,
        velocity_1,
        gravity=9.82,
        friction=0,
        redistribute_water=True,
        smooth_water=False,
        smooth_momentum=True
    ):
        self.grid = grid
        self.size = self.grid.shape
        self.cfl = cfl
        self.q0 = water - bottom
        self.q1 = self.q0 * velocity_0
        self.q2 = self.q0 * velocity_1
        self.q5 = gravity
        self.q6 = friction
        self.bottom = bottom
        self.update_bottom_gradient()
        self.redistribute_water = redistribute_water
        self.smooth_water = smooth_water
        self.smooth_momentum = smooth_momentum


    def update_bottom_gradient(self):
        d_dx = findiff.FinDiff(0, self.grid, acc=1)
        self.q3 = d_dx(self.bottom)
        self.q3 = scipy.signal.savgol_filter(
            self.q3, window_length=self.q3.shape[0] // 5 + 1, polyorder=3
        )

        d_dy = findiff.FinDiff(1, self.grid, acc=1)
        self.q4 = d_dy(self.bottom)
        self.q4 = scipy.signal.savgol_filter(
            self.q4, window_length=self.q3.shape[0] // 5 + 1, polyorder=3
        )

    def flatten(self, i, j):
        return i + self.size[0] * j

    def unflatten(self, p):
        i = p % self.size[0]
        j = (p - i) / self.size[0]
        return i, j

    def update(self, qs):
        for p, q in enumerate(qs):
            i, j = self.unflatten(p)
            self.q0[i, j] = q[0]
            self.q1[i, j] = q[1]
            self.q2[i, j] = q[2]

    def __getitem__(self, i, j):
        if i == -1:
            q = self[i + 1, j]
            q[1] *= -1
            return q
        elif i == self.size[0]:
            q = self[i - 1, j]
            q[1] *= -1
            return q
        elif j == -1:
            q = self[i, j + 1]
            q[2] *= -1
            return q
        elif j == self.size[1]:
            q = self[i, j - 1]
            q[2] *= -1
            return q
        else:
            return numpy.array([self.q0[p], self.q1[p], self.q2, self.q3[p], self.q4])

    @staticmethod
    def min_speed_deviation(q):
        return -numpy.sqrt(abs(q[5] * q[0]))

    @staticmethod
    def max_speed_deviation(q):
        return numpy.sqrt(abs(q[5] * q[0]))

    def cfl_dt(self, dt):
        for i in range(0, self.size[0] - 1):
            for j in range(0, self.size[1] -1):
                dx = self.grid[i + 1, j] - self.grid[i, j]
                dy = self.grid[i, j + 1] - self.grid[i, j]
                v0 = max(
                    abs(self.safe_velocity_0(self[i,j]) + self.max_speed_deviation(self[i,j])),
                    abs(self.safe_velocity_0(self[i,j]) + self.min_speed_deviation(self[i,j]))
                )
                v1 = max(
                    abs(self.safe_velocity_1(self[i,j]) + self.max_speed_deviation(self[i,j])),
                    abs(self.safe_velocity_1(self[i,j]) + self.min_speed_deviation(self[i,j]))
                )
                if numpy.isfinite(v0) and numpy.isfinite(v1):
                    dt = min(dt, self.cfl * max(dx, dy) / max(v0, v1))
        return dt

    def get_velocity(self):
        return self.q1 / self.q0

    def get_depth(self):
        return self.q0

    def get_bottom_gradient(self):
        return self.q3

    @staticmethod
    def clamp_velocity(v):
        return max(min(v, ShallowWater.speed), -ShallowWater.speed)

    @staticmethod
    def safe_velocity_0(q):
        v = q[1] / q[0] if q[0] > ShallowWater.dry_water else 0
        v = ShallowWater.clamp_velocity(v)
        return v

    @staticmethod
    def safe_velocity_1(q):
        v = q[2] / q[0] if q[0] > ShallowWater.dry_water else 0
        v = ShallowWater.clamp_velocity(v)
        return v

    @staticmethod
    def F0(q):
        v0 = ShallowWater.safe_velocity_0(q)
        return numpy.array([
            q[1], 
            q[1] * v0 + 0.5 * q[5] * q[0] * q[0], 
            q[2] * v0, 
            0, 
            0])

    @staticmethod
    def F1(q):
        v1 = ShallowWater.safe_velocity_1(q)
        return numpy.array([
            q[1], 
            q[1] * v1, 
            q[2] * v1 +  + 0.5 * q[5] * q[0] * q[0], 
            0, 
            0])

    @staticmethod
    def S(q):
        v0 = ShallowWater.safe_velocity_0(q)
        v1 = ShallowWater.safe_velocity_1(q)
        return numpy.array(
            [
                0,
                q[5] * q[0] * q[3] + q[6] * numpy.linalg.norm([v0, v1]) * v0,
                q[2] * q[0] * q[3] + q[4] * numpy.linalg.norm([v0, v1]) * v1,
                0,
                0,
                0,
                0,
            ]
        )

$$
\partial_t(Q) + 
\partial_0\left(
\left[
\begin{matrix}
Q_1\\
Q_1 ^2 Q_0^{-1} + 0.5 Q_5 Q_0^2\\
Q_2 Q_1 Q_0^{-1}\\
0\\
0\\
0\\
0\\
\end{matrix}
\right]
\right)
+
\partial_1\left(
\left[
\begin{matrix}
Q_2\\
Q_1 Q_2 Q_0^{-1}\\
Q_2^2 Q_0^{-1} + 0.5 Q_5 Q_0^2 \\
0\\
0\\
0\\
0\\
\end{matrix}
\right]
\right)
+\left[
\begin{matrix}
0\\
Q_5 Q_0 Q_3 + Q_6 \sqrt{(Q_1 Q_0^{-1})^2 + (Q_2 Q_0^{-1})^2} Q_1 Q_0^{-1}\\
Q_5 Q_0 Q_4 + Q_6 \sqrt{(Q_1 Q_0^{-1})^2 + (Q_2 Q_0^{-1})^2} Q_2 Q_0^{-1}\\
0\\
0\\
0\\
0\\
\end{matrix}
\right]
$$